<a href="https://colab.research.google.com/github/aryan1625/ANN_on_AQI_Dataset/blob/main/exp6multiclasslabellingandAnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from collections import Counter
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
aqi = pd.read_csv('https://archive.ics.uci.edu/static/public/360/data.csv')

In [ ]:
aqi = aqi.drop(columns=['Date', 'Time','NMHC(GT)','PT08.S1(CO)','C6H6(GT)','PT08.S2(NMHC)','PT08.S3(NOx)','PT08.S4(NO2)','RH','T','AH','NOx(GT)'])

In [ ]:
def calculateAQI(x1,x2,i):
  AQI = 0
  CO = 0
  NO2 = 0
  O3 = 0

  a = x1['CO(GT)'][i]
  b = x1['NO2(GT)'][i]
  c = x1['PT08.S5(O3)'][i]
  # x['aqi'] = val

  if(a<=2):
    CO = 1
  elif(a<=4):
    CO = 2
  elif(a<=8):
    CO = 3
  elif(a<=30):
    CO = 4
  elif(a<=100):
    CO = 5
  else:
    CO = 6


  if(b<=25):
    NO2 = 1
  elif(b<=50):
    NO2 = 2
  elif(b<=100):
    NO2 = 3
  elif(b<=200):
    NO2 = 4
  elif(b<=400):
    NO2 = 5
  else:
    NO2 = 6


  if(c<=500):
    O3 = 1
  elif(c<=1000):
    O3 = 2
  elif(c<=1300):
    O3 = 3
  elif(c<=2400):
    O3 = 4
  elif(c<=3800):
    O3 = 5
  else:
    O3 = 6
  val = round((CO+NO2+O3)/3)

  x2[i][2] = O3
  x2[i][1] = NO2
  x2[i][0] = CO
  x2[i][3] = val




In [ ]:
m = aqi.shape[0]
aqi2 = np.zeros((m, 4))
# aqi.head()
# print(aqi['aqi'])
for i in range(m):
  calculateAQI(aqi,aqi2,i)
# print(aqi2)

# # Apply the calculateAQI function to each row and unpack the returned values
# X['CO(GT)'], X['NO2(GT)'], X['PT08.S5(O3)'], y = zip(*X.apply(calculateAQI, axis=1))

# # Using .loc to update the DataFrame
# aqi.loc[:, ['CO(GT)', 'NO2(GT)', 'PT08.S5(O3)', 'aqi']] = X

# print(aqi.head())

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your dataset
# Assuming your dataset is stored in a CSV file named 'aqi_dataset.csv'
# dataset = pd.read_csv('aqi_dataset.csv')

# Splitting the dataset into independent variables (X) and dependent variable (y)
X = aqi2[:, :-1]  # All rows, all columns except the last one
y = aqi2[:, -1]   # All rows, only the last column

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = np.array(y_train).reshape(-1,1)
y_test = np.array(y_test).reshape(-1, 1)

In [ ]:
import numpy as np

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Initialize weights and biases
        self.weights_input_hidden = np.random.randn(self.input_size, self.hidden_size)
        self.biases_input_hidden = np.random.randn(1, self.hidden_size)

        self.weights_hidden_output = np.random.randn(self.hidden_size, self.output_size)
        self.biases_hidden_output = np.random.randn(1, self.output_size)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def softmax(self, x):
        exps = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exps / np.sum(exps, axis=1, keepdims=True)

    def forward_pass(self, X):
        # Input to hidden layer
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.biases_input_hidden
        self.hidden_output = self.sigmoid(self.hidden_input)

        # Hidden to output layer
        self.output_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.biases_hidden_output
        self.output = self.softmax(self.output_input)
        # self.predictions = (np.argmax(self.output, axis=1) + 1)
        max_index = np.argmax(self.output, axis=1)

        # Create a one-hot encoded version of the predictions
        self.predictions = np.zeros_like(self.output)
        self.predictions[np.arange(len(self.output)), max_index] = 1
        return self.predictions

    def backward_pass(self, X, y, output, learning_rate):
        # Calculate error
        self.output_error = y - output

        # Calculate gradients
        self.output_delta = self.output_error
        self.hidden_error = self.output_delta.dot(self.weights_hidden_output.T)
        self.hidden_delta = self.hidden_error * self.sigmoid_derivative(self.hidden_output)

        # Update weights and biases
        self.weights_hidden_output += self.hidden_output.T.dot(self.output_delta) * learning_rate
        self.biases_hidden_output += np.sum(self.output_delta, axis=0, keepdims=True) * learning_rate

        self.weights_input_hidden += X.T.dot(self.hidden_delta) * learning_rate
        self.biases_input_hidden += np.sum(self.hidden_delta, axis=0, keepdims=True) * learning_rate

    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            output = self.forward_pass(X)
            self.backward_pass(X, y, output, learning_rate)
            if epoch % 100 == 0:
                loss = np.mean(np.square(y - output))
                print(f"Epoch {epoch}, Loss: {loss}")


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# One-hot encode y_train
encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train.reshape(-1, 1))
y_test_encoded = encoder.transform(y_test.reshape(-1, 1))
print(y_test_encoded.shape)
input_size = 3
hidden_size = 10
output_size = 5
nn = NeuralNetwork(input_size, hidden_size, output_size)
# print(X_train.shape)

# Train the neural network
nn.train(X_train, y_train_encoded, epochs=1000, learning_rate=0.0001)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(1872, 5)
Epoch 0, Loss: 0.2776753507014028
Epoch 100, Loss: 0.17042084168336674
Epoch 200, Loss: 0.18752171008684035
Epoch 300, Loss: 0.07086172344689379
Epoch 400, Loss: 0.014375417501670007
Epoch 500, Loss: 0.013092852371409485
Epoch 600, Loss: 0.011436205744822979
Epoch 700, Loss: 0.011169004676018704
Epoch 800, Loss: 0.037942551770207084
Epoch 900, Loss: 0.19617902471609885


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Test the trained network
# print(X_test)
predictions = nn.forward_pass(X_test)  # No need to flatten
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Convert predictions to class labels
# predicted_labels = np.round(predictions).astype(int)

# Convert y_test to class labels
# true_labels = y_test.astype(int)
# print(y_test)
true_labels = y_test_encoded

# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy:.4f}")

# Calculate precision
precision = precision_score(true_labels, predictions, average='weighted',zero_division=1)
print(f"Precision: {precision:.4f}")

# Calculate recall
recall = recall_score(true_labels, predictions, average='weighted',zero_division=1)
print(f"Recall: {recall:.4f}")

# Calculate F1 score
f1 = f1_score(true_labels, predictions, average='weighted',zero_division=1)
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.9786
Precision: 0.9785
Recall: 0.9786
F1 Score: 0.9780
